In [ ]:
    # Scenario: AI System for Detecting Lung Diseases from Chest X-rays
    # 🚨 The Problem

    # A large hospital receives thousands of chest X-rays daily.

    # Radiologists are:

    # Overworked

    # Limited in number

    # Required to make fast decisions

    # Sometimes critical conditions like:

    # 👉 Pneumonia
    # 👉 COVID-19
    # 👉 Normal lungs

    # must be identified within minutes.

    # Delays can cost lives.

    # 💡 The Solution: Build an AI Assistant

    # The hospital decides to deploy an AI model that can pre-screen X-rays and
    # alert doctors.

    # But there is a challenge…

    # ❗ Medical datasets are usually SMALL.

    # Training a deep neural network from scratch would require:

    # Millions of labeled X-rays

    # Massive GPU clusters

    # Months of training

    # Not practical.

    # ⭐ Enter Transfer Learning (Your Code)

    # Instead of starting from zero, engineers use:

    # 👉 ResNet50 trained on ImageNet

    # Although ImageNet contains everyday objects (dogs, cars, etc.),
    # the early CNN layers learn universal visual patterns, like:

    # ✅ Edges
    # ✅ Gradients
    # ✅ Textures
    # ✅ Shapes

    # These features are also present in medical scans.

import torch
import torchvision.models as modelsd
from torch import nn

# Load pre-trained ResNet50 (trained on ImageNet)
model = models.resnet50(pretrained=True)

# Freeze all layers (don't update their weights)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer for our task (3 classes)
num_classes = 3  # Normal, Pneumonia, COVID-19
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Only the new layer will be trained
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 99.2MB/s]


Trainable params: 6147


In [ ]:
# Install required libraries if not already installed:
# pip install transformers datasets peft accelerate

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig

# 1. Load base model and tokenizer
model_name = "gpt2"   # you can replace with another causal LM
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2. Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn"]  # GPT-2 uses 'c_attn' for QKV projection
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

# 3. Create a tiny synthetic dataset
train_texts = ["Hello world", "LoRA fine-tuning is fun", "Transformers are powerful"]
eval_texts = ["Testing evaluation", "Another sample"]

train_dataset = Dataset.from_dict({"text": train_texts})
eval_dataset = Dataset.from_dict({"text": eval_texts})

# Tokenization function with labels
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=64
    )
    tokens["labels"] = tokens["input_ids"].copy()  # labels required for loss
    return tokens

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 4. Training configuration
training_args = TrainingArguments(
    output_dir="./lora-finetuned-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,   # keep small for demo
    logging_steps=1,
    save_strategy="epoch",
    fp16=True
)

# 5. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# 6. Train
trainer.train()

# 7. Save only LoRA adapter weights (few MB instead of GBs!)
model.save_pretrained("./lora-weights")

print("Training complete. LoRA weights saved in ./lora-weights")

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,8.129440


Training complete. LoRA weights saved in ./lora-weights


In [ ]:
# Big Idea Behind This Code

# Instead of shipping a 500MB–10GB model every time…

# You ship:

# 👉 A tiny adapter (few MB)

# Then dynamically attach it to the base model.

# This enables:

# ✅ Modular AI systems
# ✅ Fast deployment
# ✅ Cheap storage
# ✅ Multi-domain adapters

# This pattern is exploding across the industry.

# 🚀 Real-World Scenario (VERY Powerful)
# 🛒 Scenario: E-Commerce Company Builds Multiple AI Assistants

# An e-commerce giant wants different AI behaviors:

# Sentiment analyzer

# Product recommender

# Customer support bot

# Return-policy assistant

# But retraining separate LLMs would cost millions.

# So engineers do something smart:

# 👉 One Base Model
# 👉 Multiple LoRA Adapters


# Install required libraries if not already installed:
# pip install transformers datasets peft accelerate

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, PeftModel

# 1. Load base model and tokenizer
model_name = "gpt2"   # you can replace with another causal LM
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2. Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn"]  # GPT-2 uses 'c_attn' for QKV projection
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

# 3. Create a tiny synthetic dataset
train_texts = ["Hello world", "LoRA fine-tuning is fun", "Transformers are powerful"]
eval_texts = ["Testing evaluation", "Another sample"]

train_dataset = Dataset.from_dict({"text": train_texts})
eval_dataset = Dataset.from_dict({"text": eval_texts})

# Tokenization function with labels
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=64
    )
    tokens["labels"] = tokens["input_ids"].copy()  # labels required for loss
    return tokens

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 4. Training configuration
training_args = TrainingArguments(
    output_dir="./lora-finetuned-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=1,   # keep small for demo
    logging_steps=1,
    save_strategy="epoch",
    fp16=True
)

# 5. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# 6. Train
trainer.train()

# 7. Save only LoRA adapter weights (few MB instead of GBs!)
model.save_pretrained("lora-sentiment")  # folder will contain adapter_config.json

print("Training complete. LoRA weights saved in ./lora-sentiment")

# 8. Reload base model and attach adapter for inference
base_model = AutoModelForCausalLM.from_pretrained("gpt2")
sentiment_model = PeftModel.from_pretrained(base_model, "lora-sentiment")

# Example inference
input_text = "I love using transformers and LoRA adapters!"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
output_ids = sentiment_model.generate(input_ids, max_length=50)
print("Output:", tokenizer.decode(output_ids[0], skip_special_tokens=True))


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,8.129440


Training complete. LoRA weights saved in ./lora-sentiment


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Output: I love using transformers and LoRA adapters!

I love using transformers and LoRA adapters! I love using transformers and LoRA adapters! I love using transformers and LoRA adapters! I love using transformers and LoRA
